In [ ]:
!pip install emnist

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from emnist import extract_training_samples, extract_test_samples

In [ ]:
class RandomKernelCNN(nn.Module):
  def __init__(self, input_channels, num_classes, min_kernel_size, max_kernel_size):
    super(RandomKernelCNN, self).__init__()

    # Randomly choose kernel size between min and max for each convolutional layer
    self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=torch.randint(min_kernel_size, max_kernel_size + 1, (1,)).item(), stride=1, padding=1)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=torch.randint(min_kernel_size, max_kernel_size + 1, (1,)).item(), stride=1, padding=1)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=torch.randint(min_kernel_size, max_kernel_size + 1, (1,)).item(), stride=1, padding=1)
    self.pool = nn.AdaptiveAvgPool2d(7)
    self.fc1 = nn.Linear(64 * 7 * 7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.relu(self.conv2(x))
    x = F.relu(self.conv3(x))
    x = self.pool(x)
    x = x.view(x.size(0), -1)
    x = self.fc1(x)
    return x

In [ ]:
# Load and preprocess the EMNIST dataset
train_images, train_labels = extract_training_samples('digits')
test_images, test_labels = extract_test_samples('digits')

train_images = torch.unsqueeze(torch.tensor(train_images), 1).float() / 255.0
test_images = torch.unsqueeze(torch.tensor(test_images), 1).float() / 255.0
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Create DataLoader
train_dataset = TensorDataset(train_images, train_labels)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Instantiate the random kernel model
input_channels = 1  # grayscale images
num_classes = 47  # For balanced EMNIST dataset
min_kernel_size = 3  # Minimum kernel size
max_kernel_size = 7  # Maximum kernel size

model = RandomKernelCNN(input_channels, num_classes, min_kernel_size, max_kernel_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
epochs = 10
for epoch in range(epochs):
  for images, labels in train_dataloader:
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')


In [ ]:
# Create DataLoader for test dataset
test_dataset = TensorDataset(test_images, test_labels)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluation on test set
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_dataloader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')